# 단지코드별 1차원 데이터셋 계산구상

제외 칼럼 : 임대건물구분
기존 사용 칼럼 : 단지코드별 값이 다르지 않은 칼럼들    
( 단지코드, 총세대수, 지역, 공가수, 지하철역, 버스정류장, 단지내주차면수, 등록차량수 ) 

처리가 필요한 컬럼 
( 공급유형, 전용면적, 전용면적별세대수, 자격유형, 임대보증금, 임대료)


- 단지코드가 가지고 있는 공급유형별 개수 COUNT : 10개 -> 칼럼 10개 추가
- 단지코드별 전용면적에 대해 10 ~ 100 까지 간격 10으로 하여 개수 Count : 9개 -> 칼럼 9개 추가 
- 자격유형별 개수 Count : 15가지 -> 칼럼 15개 추가 
- 전용면적별세대수는 계산되는 전용면적구분별 총계
- 임대보증금과 임대료는 단지코드별 평균처리 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import warnings 
from matplotlib import rc

warnings.filterwarnings('ignore')
plt.rcParams['axes.unicode_minus'] = False 
rc('font', family = 'NanumMyeongjo')
pd.set_option('display.max_columns', None)
%matplotlib inline 

In [3]:
train = pd.read_csv('parking_data/train.csv')
test = pd.read_csv('parking_data/test.csv')

In [72]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2952 entries, 0 to 2951
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   단지코드                          2952 non-null   object 
 1   총세대수                          2952 non-null   int64  
 2   임대건물구분                        2952 non-null   object 
 3   지역                            2952 non-null   object 
 4   공급유형                          2952 non-null   object 
 5   전용면적                          2952 non-null   float64
 6   전용면적별세대수                      2952 non-null   int64  
 7   공가수                           2952 non-null   float64
 8   자격유형                          2952 non-null   object 
 9   임대보증금                         2383 non-null   object 
 10  임대료                           2383 non-null   object 
 11  도보 10분거리 내 지하철역 수(환승노선 수 반영)  2741 non-null   float64
 12  도보 10분거리 내 버스정류장 수            2948 non-null   float64
 13  단지내

In [9]:
train.단지코드.value_counts()

C1439    48
C2132    47
C1859    40
C1790    33
C2597    32
         ..
C1718     2
C2437     1
C2188     1
C2520     1
C2340     1
Name: 단지코드, Length: 423, dtype: int64

In [13]:
except_D = train[train['자격유형'] != 'D']

In [15]:
except_D.단지코드.value_counts()

C2051    23
C2538    19
C2507    18
C2506    17
C1177    16
         ..
C2571     1
C2520     1
C2340     1
C2188     1
C2437     1
Name: 단지코드, Length: 423, dtype: int64

In [16]:
except_D[except_D['단지코드'] == 'C2051']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2426,C2051,1164,아파트,세종특별자치시,공공임대(10년),59.39,2,0.0,A,39000000,440000,0.0,3.0,755.0,780.0
2427,C2051,1164,아파트,세종특별자치시,공공임대(10년),59.87,48,0.0,A,39000000,440000,0.0,3.0,755.0,780.0
2428,C2051,1164,아파트,세종특별자치시,공공임대(10년),59.87,241,0.0,A,39000000,440000,0.0,3.0,755.0,780.0
2429,C2051,1164,아파트,세종특별자치시,공공임대(10년),59.96,3,0.0,A,39000000,440000,0.0,3.0,755.0,780.0
2430,C2051,1164,아파트,세종특별자치시,공공임대(10년),65.39,24,0.0,A,39000000,480000,0.0,3.0,755.0,780.0
2431,C2051,1164,아파트,세종특별자치시,공공임대(10년),72.82,64,0.0,A,49000000,520000,0.0,3.0,755.0,780.0
2432,C2051,1164,아파트,세종특별자치시,공공임대(10년),74.55,267,0.0,A,49000000,535000,0.0,3.0,755.0,780.0
2433,C2051,1164,아파트,세종특별자치시,공공임대(10년),79.61,7,0.0,A,55000000,570000,0.0,3.0,755.0,780.0
2434,C2051,1164,아파트,세종특별자치시,공공임대(10년),79.74,35,0.0,A,55000000,570000,0.0,3.0,755.0,780.0
2435,C2051,1164,아파트,세종특별자치시,공공임대(10년),79.93,3,0.0,A,55000000,570000,0.0,3.0,755.0,780.0


자격유형이 D가 아닌 경우에도 단지코드별 가지는 데이터 수가 다름.

In [36]:
train[train.단지코드 == 'C1004']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
764,C1004,521,아파트,충청남도,영구임대,39.30,240,3.0,C,4682000,104180,NaN,2.0,153.0,93.0
765,C1004,521,아파트,충청남도,영구임대,39.69,264,3.0,C,4728000,105210,NaN,2.0,153.0,93.0
766,C1004,521,상가,충청남도,임대상가,19.00,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
767,C1004,521,상가,충청남도,임대상가,14.10,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
768,C1004,521,상가,충청남도,임대상가,19.31,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
769,C1004,521,상가,충청남도,임대상가,21.19,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
770,C1004,521,상가,충청남도,임대상가,21.19,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
771,C1004,521,상가,충청남도,임대상가,22.95,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
772,C1004,521,상가,충청남도,임대상가,23.40,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0
773,C1004,521,상가,충청남도,임대상가,23.40,1,3.0,D,NaN,NaN,NaN,2.0,153.0,93.0


In [62]:
counts = train.groupby(['단지코드']).nunique()

In [65]:
counts

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,,,,,,,,,
C1000,1,1,1,1,1,5,6,1,1,3,3,1,1,1,1
C1004,1,1,2,1,2,15,3,1,2,2,2,0,1,1,1
C1005,1,1,1,1,1,3,3,1,1,3,3,0,1,1,1
C1013,1,1,1,1,1,4,5,1,1,3,3,1,1,1,1
C1014,1,1,1,1,1,6,7,1,1,4,4,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2663,1,1,1,1,1,5,6,1,1,4,4,1,1,1,1
C2666,1,1,1,1,1,4,6,1,1,2,2,1,1,1,1
C2670,1,1,1,1,1,4,3,1,1,3,3,1,1,1,1


In [63]:
counts[counts['등록차량수']>1] # 단지코드별 등록차량수의 값은 일정 

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,,,,,,,,,


In [64]:
counts[counts['공급유형']>1]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
단지코드,,,,,,,,,,,,,,,
C1004,1,1,2,1,2,15,3,1,2,2,2,0,1,1,1
C1039,1,1,1,1,3,9,11,1,2,7,6,1,1,1,1
C1056,1,1,2,1,2,3,2,1,2,2,2,1,1,1,1
C1109,1,1,2,1,2,17,3,1,2,2,2,1,1,1,1
C1206,1,1,2,1,2,5,2,1,2,2,2,1,1,1,1
C1258,1,1,1,1,2,6,6,1,2,4,4,1,1,1,1
C1341,1,1,1,1,2,3,8,1,2,3,3,1,1,1,1
C1344,1,1,1,1,2,6,6,1,1,6,6,1,1,1,1
C1350,1,1,1,1,3,7,12,1,2,6,6,0,1,1,1


In [73]:
train[train['단지코드'] == 'C1039']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2100,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000,-,0.0,3.0,673.0,645.0
2101,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000,-,0.0,3.0,673.0,645.0
2102,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000,-,0.0,3.0,673.0,645.0
2103,C1039,790,아파트,서울특별시,국민임대,26.68,26,13.0,A,15282000,190620,0.0,3.0,673.0,645.0
2104,C1039,790,아파트,서울특별시,국민임대,36.70,140,13.0,A,21024000,256040,0.0,3.0,673.0,645.0
2105,C1039,790,아파트,서울특별시,국민임대,36.70,14,13.0,A,21024000,256040,0.0,3.0,673.0,645.0
2106,C1039,790,아파트,서울특별시,국민임대,36.87,43,13.0,A,21024000,256040,0.0,3.0,673.0,645.0
2107,C1039,790,아파트,서울특별시,국민임대,46.76,192,13.0,A,45456000,356420,0.0,3.0,673.0,645.0
2108,C1039,790,아파트,서울특별시,국민임대,46.96,25,13.0,A,45456000,356420,0.0,3.0,673.0,645.0
2109,C1039,790,아파트,서울특별시,영구임대,26.68,70,13.0,I,11490000,94820,0.0,3.0,673.0,645.0


In [67]:
train[train['공급유형']=='장기전세']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2043,C1397,370,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0
2044,C1397,370,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2045,C1397,370,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2046,C1397,370,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2047,C1397,370,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2048,C1397,370,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0
2100,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000,-,0.0,3.0,673.0,645.0
2101,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000,-,0.0,3.0,673.0,645.0
2102,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000,-,0.0,3.0,673.0,645.0


공급유형이 장기전세인 경우 임대료 - 로 되어 있음. 장기전세로 매달 내는 월세가 없음. 

In [68]:
train[train['임대료'] =='-']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2043,C1397,370,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0
2044,C1397,370,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2045,C1397,370,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2046,C1397,370,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2047,C1397,370,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2048,C1397,370,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0
2100,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000,-,0.0,3.0,673.0,645.0
2101,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000,-,0.0,3.0,673.0,645.0
2102,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000,-,0.0,3.0,673.0,645.0
2547,C1326,1934,아파트,부산광역시,국민임대,24.72,472,43.0,H,-,-,0.0,4.0,1670.0,1153.0


In [70]:
train.공급유형.value_counts()

국민임대         1758
임대상가          562
행복주택          213
공공임대(10년)     205
영구임대          152
공공임대(50년)      31
공공임대(분납)       12
장기전세            9
공공분양            7
공공임대(5년)        3
Name: 공급유형, dtype: int64

In [71]:
train[train['공급유형']=='장기전세']

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2043,C1397,370,아파트,서울특별시,장기전세,23.32,51,9.0,A,87444000,-,0.0,3.0,1590.0,1595.0
2044,C1397,370,아파트,서울특별시,장기전세,46.79,4,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2045,C1397,370,아파트,서울특별시,장기전세,46.91,69,9.0,A,174888000,-,0.0,3.0,1590.0,1595.0
2046,C1397,370,아파트,서울특별시,장기전세,51.96,24,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2047,C1397,370,아파트,서울특별시,장기전세,51.99,80,9.0,A,194562000,-,0.0,3.0,1590.0,1595.0
2048,C1397,370,아파트,서울특별시,장기전세,59.93,142,9.0,A,216423000,-,0.0,3.0,1590.0,1595.0
2100,C1039,790,아파트,서울특별시,장기전세,51.32,126,13.0,A,187694000,-,0.0,3.0,673.0,645.0
2101,C1039,790,아파트,서울특별시,장기전세,59.88,49,13.0,A,213863000,-,0.0,3.0,673.0,645.0
2102,C1039,790,아파트,서울특별시,장기전세,59.94,75,13.0,A,213863000,-,0.0,3.0,673.0,645.0
